# Calibrate cameras, create initial orthoimage and partial DSM

In [ ]:
import os
from glob import glob
import subprocess
import numpy as np
from tqdm import tqdm
import pandas as pd
import cv2
from shapely.geometry import Point
import geopandas as gpd
import shutil
import ast
# Ignore warnings (rasterio throws a warning whenever an image is not georeferenced. Annoying in this case.)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

# Locate image files
data_folder = '/Users/rdcrlrka/Research/Soo_locks'
image_folder = os.path.join(data_folder, '20251001_imagery', 'frames_IR')
image_list = sorted(glob(os.path.join(image_folder, '*.tiff')))
print(f"{len(image_list)} images located")

# Grab other input files
refdem_file = os.path.join(os.getcwd(), '..', 'inputs', '20251001_Soo_Model_1cm_mean_UTM19N-fake_filled_cropped.tif')
gcp_folder = os.path.join(os.getcwd(), '..', 'inputs', 'gcp')
cams_file = os.path.join(os.getcwd(), '..', 'inputs', 'cams_lonlat-fake.txt')

# Define output folders
out_folder = image_folder + '_proc_out'
os.makedirs(out_folder, exist_ok=True)
new_image_folder = os.path.join(out_folder, 'single_band_images')
undistorted_folder = os.path.join(out_folder, 'undistorted_images_cams')
init_ortho_folder = os.path.join(out_folder, 'init_ortho')
init_stereo_folder = os.path.join(out_folder, 'init_stereo')
ba_folder = os.path.join(out_folder, 'bundle_adjust')
final_ortho_folder = os.path.join(out_folder, 'final_ortho')
final_stereo_folder = os.path.join(out_folder, 'final_stereo')
full_cam_folder = os.path.join(out_folder, 'full_optimized_cams')

## Merge GCP

In [ ]:
gcp_merged_file = os.path.join(gcp_folder, 'GCP_merged.csv')
if not os.path.exists(gcp_merged_file):

    gcp_list = sorted(glob(os.path.join(gcp_folder, '*.gcp')))
    df_list = []
    for gcp_file in gcp_list:
        df = pd.read_csv(
            gcp_file,
            sep=',',
            header=None,
            skiprows=[0],
            names=[
                'point_index', 'lat', 'lon', 'Z', 'lat_sigma', 'lon_sigma', 'Z_sigma', 
                'image_path', 'col_sample', 'row_sample', 'use_lat', 'use_lon']
        )
        df_list += [df]

    dfs = pd.concat(df_list).reset_index(drop=True)

    # reproject to UTM zone 19N
    gdf = gpd.GeoDataFrame(
        dfs,
        geometry=[Point(x,y) for x,y in dfs[['lon', 'lat']].values],
        crs="EPSG:4326"
    )
    gdf = gdf.to_crs("EPSG:32619")
    gdf['X'] = [x.coords.xy[0][0] for x in gdf['geometry']]
    gdf['Y'] = [x.coords.xy[1][0] for x in gdf['geometry']]

    # use just the image file name
    gdf['image_name'] = [os.path.basename(x) for x in gdf['image_path']]

    # select relevant columns
    gdf = gdf[['image_name', 'X', 'Y', 'Z', 'col_sample', 'row_sample']]

    # save to file
    gdf.to_csv(gcp_merged_file, sep=',', index=False)
    print('Saved merged GCP:', gcp_merged_file)
else:
    print('Merged GCP already exists in file, skipping merge.')


# Reproject from UTM zone 19 N to ECEF for use in ASP
gcp_merged_ecef_file = os.path.join(gcp_folder, 'GCP_merged_ECEF.csv')
if not os.path.exists(gcp_merged_ecef_file):
    # Load the merged file
    gcp_merged = pd.read_csv(gcp_merged_file, sep=',')

    # Reproject from UTM to ECEF
    geom = [Point(x,y,z) for x,y,z in gcp_merged[['X', 'Y', 'Z']].values]
    gdf = gpd.GeoDataFrame(geometry=geom, crs="EPSG:32619")
    gdf = gdf.to_crs("EPSG:4978")
    gcp_merged['X'] = [x.x for x in gdf['geometry']]
    gcp_merged['Y'] = [x.y for x in gdf['geometry']]
    gcp_merged['Z'] = [x.z for x in gdf['geometry']]

    # Save to file
    gcp_merged.to_csv(gcp_merged_ecef_file, sep=',', index=False)
    print('Saved merged GCP in ECEF coordinates:', gcp_merged_ecef_file)
else:
    print('Merged GCP in ECEF coordinates already exists in file, skipping reprojection.')


## Convert images to single band in case they're RGB

A couple IR images (near the windows) were captured in RGB 

In [ ]:
os.makedirs(new_image_folder, exist_ok=True)

# iterate over images
print('Saving single-band images to:', new_image_folder)
for image_file in tqdm(image_list):
    # convert images to single band
    out_fn = os.path.join(new_image_folder, os.path.basename(image_file))
    if os.path.exists(out_fn):
        continue
    cmd = [
        "gdal_translate",
        "-b", "1",
        image_file, out_fn
    ]
    subprocess.run(cmd)

## Calibrate cameras using GCP

In [ ]:
def save_tsai(tsai_dict, filename):
    """
    Save a TSAI (.tsai) pinhole camera file from a dictionary.
    """
    with open(filename, 'w') as f:
        f.write("VERSION_4\n")
        f.write("PINHOLE\n")
        f.write(f"fu = {tsai_dict['fu']}\n")
        f.write(f"fv = {tsai_dict['fv']}\n")
        f.write(f"cu = {tsai_dict['cu']}\n")
        f.write(f"cv = {tsai_dict['cv']}\n")
        f.write(f"u_direction = {' '.join(map(str, tsai_dict['u_direction']))}\n")
        f.write(f"v_direction = {' '.join(map(str, tsai_dict['v_direction']))}\n")
        f.write(f"w_direction = {' '.join(map(str, tsai_dict['w_direction']))}\n")
        f.write(f"C = {' '.join(map(str, tsai_dict['C']))}\n")
        f.write("R = " + " ".join(map(str, tsai_dict['R'].flatten())) + "\n")
        f.write(f"pitch = {tsai_dict['pitch']}\n")
        # Add small distortion for bundle adjust
        f.write("TSAI\n")
        f.write("k1 = -1e-6\nk2 = 1e-6\np1 = 0\np2 = 0\nk3 = 1e-6\n")


def calibrate_cameras(image_files, gcp_file, output_folder=None, file_prefix=None, plot_results=True):
    object_points_list = []
    image_points_list = []
    image_size = None

    # --- Load merged GCP ---
    gcp = pd.read_csv(gcp_file, sep=',')    

    # --- Compile GCP (object) and image (pixel) points --- 
    for image_file in image_files:
        # Subset GCP to image
        gcp_image = gcp.loc[gcp['image_name']==os.path.basename(image_file)]

        # Object and image points
        obj_pts = gcp_image[['X','Y','Z']].values.astype(np.float32)
        image_pts = gcp_image[['col_sample','row_sample']].values.astype(np.float32)
        obj_pts = obj_pts.reshape(-1,1,3)
        image_pts = image_pts.reshape(-1,1,2)

        object_points_list.append(obj_pts)
        image_points_list.append(image_pts)

        # get image size
        if image_size is None:
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
            image_size = (image.shape[1], image.shape[0])

    if len(object_points_list) == 0:
        raise ValueError("No valid images for calibration")
    
    # --- Subtract the mean of all object points for better calculation ---
    all_obj_pts = np.vstack([op.reshape(-1,3) for op in object_points_list])
    object_points_mean = all_obj_pts.mean(axis=0)
    object_points_list = [x - object_points_mean for x in object_points_list]

    # --- Initialize intrinsics --- 
    fx = fy = 2000
    cx = image_size[0] / 2
    cy = image_size[1] / 2
    K_init = np.array([
        [fx,0,cx],
        [0,fy,cy],
        [0,0,1]
        ], dtype=np.float64)
    dist_init = np.zeros(8)
    flags = (
        cv2.CALIB_USE_INTRINSIC_GUESS
        | cv2.CALIB_FIX_PRINCIPAL_POINT 
        | cv2.CALIB_ZERO_TANGENT_DIST
        )

    # --- Calibrate cameras ---
    rms, K, dist, rvecs, tvecs = cv2.calibrateCamera(
        object_points_list,
        image_points_list,
        image_size,
        K_init,
        dist_init,
        flags=flags
    )

    print("Shared calibration done")
    print("RMS reprojection error:", rms)
    print("Camera matrix K:\n", K)
    print("Distortion coefficients:", dist.ravel())

    # --- Calculate adjusted camera matrix ---
    w,h = image_size
    K_full, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w,h), 1, (w, h))

    # --- Save camera calibration parameters ---
    calib_file = os.path.join(output_folder, file_prefix + 'camera_calibration_params.csv')
    calib_df = pd.DataFrame({
        'image_name': image_files,
        'K': [[float(x) for x in K.ravel()]]*len(image_files),
        'K_full': [[float(x) for x in K_full.ravel()]]*len(image_files),
        'distortion_coefficients': [[float(x) for x in dist.ravel()]]*len(image_files),
        'RMS': [rms]*len(image_files)
    })
    calib_df.to_csv(calib_file, index=False)
    print("Saved calibration params:", calib_file)

    # --- Save undistorted images, GCP, and camera extrinsics ---
    print('Estimating individual image extrinsics')
    for i, image_file in enumerate(tqdm(image_files)):
        # Undistort image
        image_undistorted_file = os.path.join(
            output_folder, 
            os.path.splitext(os.path.basename(image_file))[0] + '_undistorted.tiff'
            )
        image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        # Save one cropped and one not cropped
        image_undistorted = cv2.undistort(image, K, dist, None, K)
        cv2.imwrite(image_undistorted_file, image_undistorted)
        image_undistorted_nocrop = cv2.undistort(image, K, dist, None, K_full)
        cv2.imwrite(image_undistorted_file.replace('.tiff', '_nocrop.tiff'), image_undistorted_nocrop)

        # Undistort GCP pixel coordinates
        gcp_image = gcp.loc[gcp['image_name']==os.path.basename(image_file)]
        image_pts = gcp_image[['col_sample','row_sample']].values.astype(np.float32)
        undistorted_pts = cv2.undistortPoints(image_pts, K, dist, P=K).reshape(-1, 2)
        gcp_image['col_sample_undistorted'] = undistorted_pts[:, 0]
        gcp_image['row_sample_undistorted'] = undistorted_pts[:, 1]
        # reproject to lat-lon
        gcp_reformat = gcp_image.copy()
        gcp_reformat['geometry'] = [Point(x,y) for x,y in gcp_image[['X','Y']].values]
        gcp_gdf = gpd.GeoDataFrame(geometry=gcp_reformat['geometry'], crs='EPSG:4978')
        gcp_gdf = gcp_gdf.to_crs("EPSG:4326")
        gcp_reformat['lon'] = [x.coords.xy[0][0] for x in gcp_gdf['geometry']]
        gcp_reformat['lat'] = [x.coords.xy[1][0] for x in gcp_gdf['geometry']]
        # update image names
        gcp_reformat['image_name'] = [x.replace('.tiff','_undistorted.tiff') for x in gcp_reformat['image_name']]
        # add other relevant columns
        gcp_reformat[['lat_std', 'lon_std', 'Z_std']] = 0.2, 0.2, 0.2
        gcp_reformat[['use_lat', 'use_lon']] = 1, 1
        # reorder and select appropriate columns
        gcp_reformat = gcp_reformat[[
            'lat', 'lon', 'Z', 'lat_std', 'lon_std', 'Z_std', 
            'image_name', 'col_sample_undistorted', 'row_sample_undistorted',
            'use_lat', 'use_lon'
            ]]
        gcp_reformat.reset_index(drop=True, inplace=True)
        # save to file
        gcp_undistorted_file = os.path.join(
            output_folder, 
            os.path.splitext(os.path.basename(image_file))[0] + '_undistorted.gcp'
            )
        gcp_reformat.to_csv(
            gcp_undistorted_file, 
            sep=' ',
            index=True,
            header=False
            )

        # Save camera extrinsics as TSAI camera model
        # convert rotation matrix from world -> camera to camera -> world
        R_wc = cv2.Rodrigues(rvecs[i])[0]
        R_cw = R_wc.T
        # calculate camera center
        C = -R_cw @ tvecs[i].reshape(3) + object_points_mean.reshape(3)
        # compile in dictionary
        tsai_dict = {
            'fu': K[0,0],
            'fv': K[1,1],
            'cu': K[0,2],
            'cv': K[1,2],
            'u_direction': [1,0,0],
            'v_direction': [0,1,0],
            'w_direction': [0,0,1],
            'C': C,
            'R': R_cw,
            'pitch': 1,
        }
        # save to file
        tsai_file = os.path.join(
            output_folder, 
            os.path.splitext(os.path.basename(image_file))[0] + '_undistorted.tsai'
            )
        save_tsai(tsai_dict, tsai_file)

        if plot_results:
            fig, ax = plt.subplots(1, 2, figsize=(14, 5))
            ax[0].imshow(image, cmap='Grays_r')
            ax[0].plot(
                gcp_image['col_sample'], gcp_image['row_sample'], 'xr',
                markersize=5, linewidth=1.5
                )
            ax[0].set_title('Original')
            ax[1].imshow(image_undistorted, cmap='gray')
            ax[1].plot(
                gcp_image['col_sample_undistorted'], gcp_image['row_sample_undistorted'], 'xr',
                markersize=5, linewidth=1.5
                )
            ax[1].set_title('Undistorted')
            for axis in ax:
                axis.set_xticks([]), axis.set_yticks([])
            plt.tight_layout()
            # save to file
            fig_file = os.path.join(output_folder, os.path.splitext(os.path.basename(image_file))[0] + '_undistorted.png')
            fig.savefig(fig_file, dpi=300, bbox_inches='tight')
            # print('Saved results figure:', fig_file)
            plt.close()

    return

os.makedirs(undistorted_folder, exist_ok=True)

# GROUP 1
print('\nGROUP 1: ch01-08\n----------')
image_list = sorted(glob(os.path.join(new_image_folder, '*.tiff')))[0:8]
# calculate shared calibration
print('Optimizing shared camera intrinsics using GCP...')
calibrate_cameras(image_list, gcp_merged_ecef_file, undistorted_folder, file_prefix='group1-')

# GROUP 2
print('\nGROUP 2: ch09-16\n----------')
image_list = sorted(glob(os.path.join(new_image_folder, '*.tiff')))[8:]
# calculate shared calibration
print('Optimizing shared camera intrinsics using GCP...')
calibrate_cameras(image_list, gcp_merged_ecef_file, undistorted_folder, file_prefix='group2-')

## Initial orthorectification

In [ ]:
os.makedirs(init_ortho_folder, exist_ok=True)

image_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tiff')))
cam_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tsai')))

# Mapproject
for image_file, cam_file in zip(image_list, cam_list):
    image_out_file = os.path.join(init_ortho_folder, os.path.basename(image_file).replace('.tiff', '_map.tiff'))
    cmd = [
        'mapproject',
        '--threads', '12',
        '--nodata-value', '0',
        '--tr', '0.003',
        refdem_file, image_file, cam_file, image_out_file
    ]
    subprocess.run(cmd)

# Mosaic orthoimages
print('\nMosaicking orthoimages')
image_list = sorted(glob(os.path.join(init_ortho_folder, '*.tiff')))
mosaic_file = os.path.join(init_ortho_folder, f'orthomosaic.tif')
fnc = shutil.which('gdal_merge.py')
cmd = [
    'python', fnc,
    '-o', mosaic_file,
    '-n', '0',
    '-a_nodata', '-9999'
] + image_list
subprocess.run(cmd)

## Run stereo preprocessing to create dense match files

In [ ]:
init_stereo_folder = os.path.join(out_folder, 'init_stereo')
os.makedirs(init_stereo_folder, exist_ok=True)

image_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tiff')))
cam_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tsai')))

# Set up image pairs
image1_list, image2_list = image_list[0:-1], image_list[1:]
cam1_list, cam2_list = cam_list[0:-1], cam_list[1:]

# skip the 8/9 cams pair (different intrinsics solving during bundle adjust)
iskip = [i for i in range(0,len(image1_list)) if 'ch08' in image1_list[i]][0]
image1_list = image1_list[0:iskip] + image1_list[iskip+1:]
image2_list = image2_list[0:iskip] + image2_list[iskip+1:]
cam1_list = cam1_list[0:iskip] + cam1_list[iskip+1:]
cam2_list = cam2_list[0:iskip] + cam2_list[iskip+1:]

# Iterate over pairs
for i in tqdm(range(len(image1_list))):
    image1, image2 = image1_list[i], image2_list[i]
    cam1, cam2 = cam1_list[i], cam2_list[i]

    pair_prefix = os.path.join(
        init_stereo_folder,
        os.path.splitext(os.path.basename(image1))[0] + '__' + os.path.splitext(os.path.basename(image2))[0],
        'run'
        )
    
    cmd = [
        'parallel_stereo',
        '--threads-singleprocess', '12',
        '--threads-multiprocess', '12',
        '--stop-point', '1',
        image1, image2,
        cam1, cam2,
        pair_prefix,
    ]
    subprocess.run(cmd)

## Bundle adjust

In [ ]:
ba_folder = os.path.join(out_folder, 'bundle_adjust')
os.makedirs(ba_folder, exist_ok=True)

image_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tiff')))
cam_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted.tsai')))

# Copy dense matches to bundle adjust folder
match_list = sorted(glob(os.path.join(init_stereo_folder, '*', '*.match')))
for match_file in match_list:
    # get image pair
    pair = os.path.dirname(match_file).split('/')[-1]
    # check which group it's in
    first_channel = pair.split('ch')[1][0:2]
    group = 1 if float(first_channel) < 9 else 2
    # define output file
    match_out_file = os.path.join(
        ba_folder, 
        f'run_group{group}-{pair}' + '.match'
        )
    # copy
    _ = shutil.copy2(match_file, match_out_file)

# GROUP 1
print('\nGROUP 1: ch01-08\n----------')
image_list_group1 = image_list[0:8]
cam_list_group1 = cam_list[0:8]
# Run bundle adjust
cmd = [
    'parallel_bundle_adjust',
    '--threads', '12',
    '--num-iterations', '2000',
    '--num-passes', '2',
    '--inline-adjustments',
    '--force-reuse-match-files',
    '--heights-from-dem', refdem_file,
    '--heights-from-dem-uncertainty', '0.01',
    '--solve-intrinsics',
    '--intrinsics-to-share', 'optical_center,other_intrinsics',
    '--intrinsics-to-float', 'all',
    # '--fixed-distortion-indices', '2,3',
    '-o', os.path.join(ba_folder, 'run_group1')
] + image_list_group1 + cam_list_group1
subprocess.run(cmd)

# GROUP 2
print('\nGROUP 2: ch09-16\n----------')
image_list_group2 = image_list[8:]
cam_list_group2 = cam_list[8:]
# Run bundle adjust
cmd = [
    'parallel_bundle_adjust',
    '--threads', '12',
    '--num-iterations', '2000',
    '--num-passes', '2',
    '--inline-adjustments',
    '--force-reuse-match-files',
    '--heights-from-dem', refdem_file,
    '--heights-from-dem-uncertainty', '0.01',
    '--solve-intrinsics',
    '--intrinsics-to-share', 'optical_center,other_intrinsics',
    '--intrinsics-to-float', 'all',
    # '--fixed-distortion-indices', '2,3',
    '-o', os.path.join(ba_folder, 'run_group2')
] + image_list_group2 + cam_list_group2
subprocess.run(cmd)

## Modify optimized cameras to include full image

In [ ]:
def update_tsai_intrinsics_to_full_fov(
    calib_csv_list,
    ba_cam_folder,
    output_cam_folder,
):
    os.makedirs(output_cam_folder, exist_ok=True)
    cam_list = sorted(glob(os.path.join(ba_cam_folder, '*.tsai')))

    for calib_file in calib_csv_list:
        print('Processing:',calib_file)
        calib = pd.read_csv(calib_file)
        calib["K"] = calib["K"].apply(lambda s: np.array(ast.literal_eval(s)))
        calib["K_full"] = calib["K_full"].apply(lambda s: np.array(ast.literal_eval(s)))
        calib["distortion_coefficients"] = calib["distortion_coefficients"].apply(
            lambda s: np.array(ast.literal_eval(s))
        )

        for _, row in tqdm(calib.iterrows(), total=len(calib)):
            image_file = row["image_name"]
            image_base = os.path.splitext(os.path.basename(image_file))[0]

            # Find matching .tsai camera file
            cam_matches = [x for x in cam_list if image_base in os.path.basename(x)]
            if not cam_matches:
                print(f"[WARN] No camera found for {image_base}")
                continue
            cam_file = cam_matches[0]

            # Read the optimized camera intrinsics from .tsai
            with open(cam_file, "r") as f:
                cam_lines = [l.strip() for l in f.readlines() if l.strip()]

            fu = fv = cu = cv = None
            for line in cam_lines:
                if line.startswith("fu"):
                    fu = float(line.split()[-1])
                elif line.startswith("fv"):
                    fv = float(line.split()[-1])
                elif line.startswith("cu"):
                    cu = float(line.split()[-1])
                elif line.startswith("cv"):
                    cv = float(line.split()[-1])

            if None in (fu, fv, cu, cv):
                print(f"[WARN] Missing intrinsic values in {cam_file}")
                continue

            # Construct intrinsic matrices
            K_opt = np.array([[fu, 0, cu],
                              [0, fv, cv],
                              [0, 0, 1]])
            K_crop = row["K"].reshape(3, 3)
            K_full = row["K_full"].reshape(3, 3)

            # Compute transform crop -> full
            H = K_full @ np.linalg.inv(K_crop)

            # Map optimized intrinsics into full-FOV coordinate system
            K_opt_full = H @ K_opt
            K_opt_full /= K_opt_full[2, 2]

            fu_full = K_opt_full[0, 0]
            fv_full = K_opt_full[1, 1]
            cu_full = K_opt_full[0, 2]
            cv_full = K_opt_full[1, 2]

            # Update lines
            updated_lines = []
            for line in cam_lines:
                if line.startswith("fu"):
                    updated_lines.append(f"fu = {fu_full}")
                elif line.startswith("fv"):
                    updated_lines.append(f"fv = {fv_full}")
                elif line.startswith("cu"):
                    updated_lines.append(f"cu = {cu_full}")
                elif line.startswith("cv"):
                    updated_lines.append(f"cv = {cv_full}")
                else:
                    updated_lines.append(line)

            # Write out new camera file
            cam_out_file = os.path.join(
                output_cam_folder, os.path.basename(cam_file).replace(".tsai", "_full.tsai")
            )
            with open(cam_out_file, "w") as f:
                f.write("\n".join(updated_lines) + "\n")


calib_list = sorted(glob(os.path.join(undistorted_folder, '*camera_calibration_params.csv')))
update_tsai_intrinsics_to_full_fov(calib_list, ba_folder, full_cam_folder)


## Final orthorectification

In [ ]:
os.makedirs(final_ortho_folder, exist_ok=True)

image_list = sorted(glob(os.path.join(undistorted_folder, '*_undistorted_nocrop.tiff')))
cam_list = sorted(glob(os.path.join(full_cam_folder, '*.tsai')))

# Mapproject
for image_file, cam_file in zip(image_list, cam_list):
    image_out_file = os.path.join(final_ortho_folder, os.path.basename(image_file).replace('.tiff', '_map.tiff'))
    cmd = [
        'mapproject',
        '--threads', '12',
        '--nodata-value', '0',
        '--tr', '0.003',
        refdem_file, image_file, cam_file, image_out_file
    ]
    subprocess.run(cmd)

# Mosaic orthoimages
print('\nMosaicking orthoimages')
image_list = sorted(glob(os.path.join(final_ortho_folder, '*.tiff')))
mosaic_file = os.path.join(final_ortho_folder, f'orthomosaic.tif')
fnc = shutil.which('gdal_merge.py')
cmd = [
    'python', fnc,
    '-o', mosaic_file,
    '-n', '0',
    '-a_nodata', '-9999'
] + image_list
subprocess.run(cmd)


In [ ]:

test_folder = os.path.join(out_folder, 'testing')
os.makedirs(test_folder, exist_ok=True)

# Open the calibration file
calib_file = os.path.join(undistorted_folder, 'group1-camera_calibration_params.csv')
calib = pd.read_csv(calib_file)
# convert to arrays
calib["K"] = calib["K"].apply(lambda s: np.array(ast.literal_eval(s)))
calib["K_full"] = calib["K_full"].apply(lambda s: np.array(ast.literal_eval(s)))
calib['distortion_coefficients'] = calib["distortion_coefficients"].apply(lambda s: np.array(ast.literal_eval(s)))

# Open the image
image_file = calib.iloc[0]['image_name']
image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

# Read the params
K_crop = calib.iloc[0]['K'].reshape(3,3)
K_full = calib.iloc[0]['K_full'].reshape(3,3)
dist = calib.iloc[0]['distortion_coefficients'].reshape(-1,1)
h,w = image.shape

# Adjust focal length in the TSAI camera
cam_file = glob(os.path.join(ba_folder, '*' + os.path.splitext(os.path.basename(image_file))[0] + '*.tsai'))[0]
with open(cam_file, 'r') as f:
    cam_lines = f.read().split('\n')
cam_lines = [x for x in cam_lines if x!='']
for line in cam_lines:
    if 'fu' in line:
        fu = float(line.split(' ')[-1])
    if 'fv' in line:
        fv = float(line.split(' ')[-1])
    if 'cu' in line:
        cu = float(line.split(' ')[-1])
    if 'cv' in line:
        cv = float(line.split(' ')[-1])

# Construct optimized K from bundle adjust
K_opt = np.array([
    [fu, 0, cu],
    [0, fv, cv],
    [0, 0, 1]
    ])

# Calculate transform from cropped -> full undistorted image frame
H = K_full @ np.linalg.inv(K_crop)

# Map optimized intrinsics into full image coordinate system
K_opt_full = H @ K_opt
K_opt_full /= K_opt_full[2, 2]  # normalize
fu_full = K_opt_full[0, 0]
fv_full = K_opt_full[1, 1]
cu_full = K_opt_full[0, 2]
cv_full = K_opt_full[1, 2]

print(fu_full, fv_full, cu_full, cv_full)

# Write to file
cam_out_file = os.path.join(test_folder, os.path.basename(cam_file).replace('.tsai', '_full.tsai'))
for i, line in enumerate(cam_lines):
    if 'fu' in line:
        cam_lines[i] = f'fu = {fu_full}'
    if 'fv' in line:
        cam_lines[i] = f'fv = {fv_full}'
    if 'cu' in cam_lines:
        cam_lines[i] = f'cu = {cu_full}'
    if 'cv' in cam_lines:
        cam_lines[i] = f'cv = {cu_full}'
cam_lines_merged = '\n'.join(cam_lines) + '\n'
with open(cam_out_file, 'w') as f:
    f.write(cam_lines_merged)